In [8]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')
MODEL="gpt-4"

In [9]:
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
#from PIL import Image               # to load images
#from IPython.display import display # to display images

#pil_im = Image.open('/content/langchain and pinecone.png')
#display(pil_im)

In [10]:
# !pip install --upgrade langchain openai  -q

In [11]:
#!pip install unstructured -q
#!pip install unstructured[local-inference] -q
#!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q

In [12]:
#!apt-get install poppler-utils  

https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/directory_loader.html

In [13]:
from langchain.document_loaders import DirectoryLoader

directory = '../data/talmud'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

7145

https://python.langchain.com/en/latest/modules/indexes/text_splitters/getting_started.html

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

7231


In [15]:
print(docs[0].page_content)

If a son cohabits with his mother who is a harlot, promising her a sheep, that sheep may not be brought as a sacrifice. She can't sue him for the sheep in court, because of "one is subject only to the greater (death) penalty," so their deal is invalid, but the prohibition takes effect.


In [16]:
#requires for open ai embedding
#!pip install tiktoken -q

In [17]:
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [18]:
#!pip install pinecone-client -q

https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html

In [19]:
import pinecone 
from langchain.vectorstores import Pinecone
# initialize pinecone
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'), 
    environment="us-west4-gcp-free"  # next to api key in console
)

print(os.getenv('PINECONE_API_KEY'))
index_name = "langchain-demo"

#index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
index = Pinecone.from_existing_index(index_name, embeddings)

f2978d29-7bdb-450c-8d78-f6ccd57328c6


In [20]:
def get_similiar_docs(query,k=5,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = "When do you say Shema?"
similar_docs = get_similiar_docs(query)
len(similar_docs)
similar_docs[0]

Document(page_content='However, the Shema is not said when one actually goes to sleep or wakes up, but rather in the general period when people lie down and get up. When is this? In the evening - when the Kohanim, who were impure, are returning from the mikveh to eat priestly portion (terumah), that is, at nightfall. That is when the time to say the Shema in the evening begins, but when does it end? Rabbi Eliezer said, "Until the end of the first watch, that is, the first part of the night." Rabbi Eliezer understands "when you lie down" as the time when people go to sleep. The Sages say until midnight, and Rabban Gamliel says until dawn. We can understand Rabban Gamliel, \'when people lie down" means when people are asleep. But the opinion of the Sages will require clarification.', metadata={'source': '../data/talmud/brachot2.txt'})

In [21]:
from langchain.llms import OpenAI

llm = OpenAI(model_name=MODEL)

/home/mark/anaconda3/envs/OpenAI/lib/python3.10/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/mark/anaconda3/envs/OpenAI/lib/python3.10/site-packages/langchain/llms/openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


https://python.langchain.com/en/latest/use_cases/question_answering.html

In [22]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  # print(similar_docs)
  answer =  chain.run(input_documents=similar_docs, question=query)
  return  answer

query = "When to say Shema?"  
get_answer(query)

'One must say the Shema prayer in the morning, "when people rise." This is when there is enough light to distinguish between the blue and the white wool (which is used in one\'s tzitzit). Some say, one needs to be able to distinguish between a wolf and a dog, and some say to recognize one\'s friend from four steps away. The time to say the Shema ends, according to Rabbi Eliezer, at sunrise, since most people are up by then. However, Rabbi Yehoshua says it ends three hours into the day, since kings do sleep that late.'

In [23]:
query = "what are the sacrifices for? \
Sacrifices are typically brought for mistakes or unintentional transgressions, as stated in Keritot 9. \
However, there are cases when one brings an offering for intentional acts, such as relations with a slavewoman designated for another, a nazir who went to the cemetery, and one who swore a false oath of testimony (also mentioned in Keritot 9) \
what about bird sacrifices?"
get_answer(query)

'Bird sacrifices are brought as offerings in cases where the person is financially poor and has committed a transgression or mistake that requires a sacrifice. This is mentioned in Keritot 10.'